In [1]:
# Importing all needed functions
import os
import io
from contextlib import redirect_stdout
from astropy.io import fits
import glob
from Calculating_det_angles import estimate_source_angles_detectors  # Importing ma'am's function
from Tools import tools
import threading
from queue import Queue
import queue
import traceback
import pandas as pd
import os
from collections import Counter
import sys
import pandas as pd



In [2]:
data_set_path = r"C:\Users\arpan\Downloads\Uncertain\Uncertain"
df = pd.read_csv('all_events.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6372 entries, 0 to 6371
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   event_name   6372 non-null   object 
 1   event_type   6372 non-null   object 
 2   reliability  6372 non-null   float64
dtypes: float64(1), object(2)
memory usage: 149.5+ KB
None


In [3]:
# Value to remove
value_to_remove = 'SFL'

# Filter to keep rows where 'col1' is not equal to the value to remove
df = df[df['event_type'] != value_to_remove]

In [4]:
# Get the number of entries for each unique value
value_counts = df['event_type'].value_counts()

# Print the value counts
print(value_counts)

event_type
GRB       2710
SFLARE    1611
TGF       1386
SGR        492
Name: count, dtype: int64


In [5]:
df.head()

,event_name,event_type,reliability
0,bn221109533,SGR,1.0
1,bn220709626,SGR,1.0
2,bn221204313,SGR,1.0
3,bn160727543,SGR,1.0
4,bn220626269,SGR,1.0


In [6]:
# reading the ttsplit file
df = pd.read_csv('ttsplit')

# getting the test and train events
test_events = df[df['category'] == 'test']['filename'].tolist()

train_events = df[df['category'] == 'train']['filename'].tolist()

# loading the event names and event types in the list

event_names = []
event_types = []

for _,row in df.iterrows():
    if (row["category"] == "test" or row["category"] == "train") and row["filename"][-1] != 'd':
        event_type,event_name = row['filename'].split("_")
        event_types.append(event_type)
        event_names.append(event_name)

print(len(event_names))
print(len(event_types))


data_set_path = r"C:\Users\arpan\Downloads\Uncertain\Uncertain"

# Get a list of all folders in the specified directory
folders = [folder for folder in os.listdir(data_set_path) if os.path.isdir(os.path.join(data_set_path, folder))]



3492
3492


In [7]:
print(folders[0])
print(os.path.join(r"C:\Users\arpan\Downloads\Uncertain\Uncertain" , folders[0]))

BELOWHZ_bn091216359
C:\Users\arpan\Downloads\Uncertain\Uncertain\BELOWHZ_bn091216359


In [8]:

dir_path = tools.json_path(r'data_path.json')

# Function to process a single event
def process_event(folder,thread_num):
    try:
        event_name = folder.split('_')[-1]
        event = event_name
        year = '20' + event[2:4] + "/"

        # URL of the file you want to download : TRIGDAT file
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        
        tools.run_wget_download(url, os.path.join(r"C:\Users\arpan\Downloads\Uncertain\Uncertain" , folder))

        print(event_queue.qsize()) 

    except Exception as e:
        # Get formatted traceback
        traceback_str = traceback.format_exc()

        # Access exception attributes (example)
        if isinstance(e, ZeroDivisionError):
            error_details = "Division by zero attempted."
        else:
            error_details = str(e.args[0]) if len(e.args) else "Unknown error."

        # Custom message based on exception type
        if isinstance(e, ValueError):
            custom_message = "Invalid value provided. Check your input data."
        else:
            custom_message = "An error occurred during processing."

        # Print detailed message
        print(f"""Error:
        - Event: {event_name}
        - Exception: {type(e).__name__}
        - Message: {e}
        - Details: {error_details}
        - Traceback:
        {traceback_str}
        - {custom_message}""")

# Create a queue to store the events
event_queue = Queue()

# Function to run the worker threads
def worker(thread_num):
    print(f"Worker thread {thread_num} started.")

    while True : #
        # Get an event from the queue
        folder = event_queue.get()
        print(f"Worker thread {thread_num} processing event {folder}.")

        # Process the event
        process_event(folder,thread_num)

        # Signal that the task is completed
        event_queue.task_done()
        print(f"Worker thread {thread_num} processed event {folder}.")

        if event_queue.qsize() == 0:
            break
    
    print(f"Worker thread {thread_num} finished.")
    
# Create and start the worker threads
num_threads = 16
threads = []
for i in range(num_threads):
    worker_thread = threading.Thread(target=worker, args=(i,), daemon=True)
    worker_thread.start()
    threads.append(worker_thread)

# Add events to the queue
for folder in folders:
    event_queue.put(folder)

print(event_queue.qsize()) 

# # Wait for all events to be processed
# event_queue.join()
# print("All events have been processed.")

# Wait for all threads to finish
for thread in threads:
    thread.join()

print('done')

Worker thread 0 started.
Worker thread 1 started.
Worker thread 2 started.
Worker thread 3 started.
Worker thread 4 started.
Worker thread 5 started.
Worker thread 6 started.
Worker thread 7 started.
Worker thread 8 started.
Worker thread 9 started.
Worker thread 10 started.
Worker thread 11 started.
Worker thread 12 started.
Worker thread 13 started.
Worker thread 14 started.
Worker thread 15 started.
90
Worker thread 0 processing event BELOWHZ_bn091216359.
download start
Worker thread 1 processing event BELOWHZ_bn110808375.
download start
Worker thread 2 processing event BELOWHZ_bn120315329.
download start
Worker thread 3 processing event BELOWHZ_bn120605953.
download start
Worker thread 4 processing event BELOWHZ_bn120617678.
download start
Worker thread 5 processing event BELOWHZ_bn121115044.
download start
Worker thread 6 processing event BELOWHZ_bn130628931.
download start
Worker thread 7 processing event BELOWHZ_bn130816205.
download start
Worker thread 8 processing event BELOWH